In [85]:
import numpy as np
import pandas as pd
import glob
import os
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy.table import Table,join
import itertools
from astroquery.simbad import Simbad
from utils import PLOT_PARAMS
PLOT_PARAMS()
from datetime import date
today=date.today()
DATE =today.strftime("%m_%d_%y")
print(DATE)
import pickle
import thejoker as tj
import pymc as pm
import corner
import arviz as az
plt.rcParams['figure.facecolor'] = 'white'
import warnings
warnings.filterwarnings("ignore")

08_08_25


In [2]:
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
import astropy.units as u

vlt = EarthLocation.of_site('paranal')  # the easiest way... but requires internet

In [3]:
files_p112 = glob.glob('../data/P112_data/*.fits')
files_p113 = glob.glob('../data/P113_data/*.fits')


In [4]:
def get_galah():
    galah =(Table.read('../../lithium/data/GALAH_DR3_main_allstar_v2.fits',format='fits'))#.to_pandas()

    dr2_ids = np.array(galah['dr2_source_id']).astype('str')
    dr3_ids = np.array(galah['dr3_source_id']).astype('str')
    
    galah.remove_column('dr2_source_id')
    galah.remove_column('dr3_source_id')
    galah = galah.to_pandas()

    galah.insert(2, 'dr2_source_id', dr2_ids)
    galah.insert(3, 'dr3_source_id', dr3_ids)
    
    galah['ph_qual_wise']=galah['ph_qual_wise'].str.decode("utf-8")
    return galah
galah=get_galah()

In [5]:
galah_rv = Table.read('/Users/maryumsayeed/Desktop/Code/Research/lithium/data/GALAH_DR3_VAC_rv_v2.fits').to_pandas()

In [6]:
def get_dir(files):
    obj_dir = {}

    counts = []
    for i,file in enumerate(files):
        with fits.open(file) as hdul:
            hdul = fits.open(file)
            header  = hdul[0].header
            #print(header.cards['HIERARCH ESO QC CCF RV ERROR'])
            data    = hdul[1].data
            time    = header['HIERARCH ESO QC BJD']
            name    = header['OBJECT']
            obj     = int(header['HIERARCH ESO OBS NAME'].split('-')[0][2:])
            snr     = header['SNR']
            counts.append(obj)

            grade   = header['OB_GRADE'].replace(" ", "")  
            if grade =='C':
                print('RG',obj,header['OB_GRADE'])
                continue

            ra,dec  = header['RA'], header['DEC']
            obs_T   = header['DATE-OBS'].split('T')[0]
            rv_val  =  header['HIERARCH ESO QC CCF RV']#header['HIERARCH ESO OCS OBJ RV'] #km/s
            rv_err  = header['HIERARCH ESO QC CCF RV ERROR'] #km/s
            sc = SkyCoord(ra=ra*u.deg, dec=dec*u.deg)
            rv_corr = rv_val
            err_corr = rv_err

            if obj in obj_dir.keys():
                obj_dir[obj]['rv'].append(rv_corr)
                obj_dir[obj]['time'].append(time)
                obj_dir[obj]['err'].append(err_corr)
                obj_dir[obj]['name'].append(name)
                obj_dir[obj]['grade'].append(grade)
                obj_dir[obj]['snr'].append(snr)

            else:
                obj_dir[obj] = {'rv': [rv_corr], 'time': [time], 'err': [err_corr], 
                                'name': [name], 'grade': [grade], 'snr': [snr]}
          
    targets_to_remove = []
    for k, v in obj_dir.items():
        if len(v['time']) < 3:
            targets_to_remove.append(k)
    for i in range(len(targets_to_remove)):
        del obj_dir[targets_to_remove[i]]
    print(len(obj_dir.keys()))

    myKeys = list(obj_dir.keys())
    myKeys.sort()
    obj_dir = {i: obj_dir[i] for i in myKeys}

    return obj_dir

obj_dir_p112 = get_dir(files_p112)
del obj_dir_p112[13] #13 ended up being a dipper star
obj_dir_p113 = get_dir(files_p113)

RG 20 C
RG 13 C
RG 14 C
RG 1 C
10
RG 12 C
RG 6 C
RG 3 C
RG 11 C
RG 22 C
RG 15 C
RG 3 C
RG 8 C
RG 3 C
RG 25 C
RG 8 C
RG 22 C
RG 3 C
RG 7 C
RG 23 C
25


In [7]:
def combine_table(obj_dir,program):
    new_df = {'RG_id':[], 'galah_id': [], 'obj_name': [], 'esp_time':[],'esp_rv':[],'esp_rv_err':[],
              'program':[]
             }

    for k, v in obj_dir.items():
        if len(v['rv']) < 3: 
            continue
        result_table = Simbad.query_objectids(v['name'][0])
        result_table = np.array(result_table).astype('str')

        for i in result_table:
            if 'GALAH' in i: 
                galah_name = i.split(' ')[-1]

        new_df['RG_id'].append(k)
        new_df['program'].append(program)
        new_df['galah_id'].append(int(galah_name))
        new_df['obj_name'].append(v['name'][0])
        new_df['esp_rv'].append(v['rv'])
        new_df['esp_time'].append(v['time'])
        new_df['esp_rv_err'].append(v['err'])
        
    new_df = pd.DataFrame(new_df)

    cols = ['sobject_id', 'vbroad', 'fe_h', 'e_fe_h' ,'Li_fe', 'e_Li_fe','ruwe_dr2','e_vbroad', 'Ba_fe', 'e_Ba_fe','La_fe','e_La_fe','Zr_fe', 'e_Zr_fe','Y_fe','e_Y_fe']
    new_df = pd.merge(new_df, galah[cols], left_on='galah_id', right_on='sobject_id')
    new_df['Li_val'] = new_df.fe_h+new_df.Li_fe+1.05
    new_df['e_Li_val'] = np.sqrt(new_df.e_fe_h**2.+new_df.e_Li_fe**2.)

    cols = ['RG_id', 'galah_id', 'obj_name', 
            'esp_time','esp_rv','esp_rv_err',
            'Li_val', 'e_Li_val','vbroad','e_vbroad', 'fe_h','ruwe_dr2', 
            'Ba_fe', 'e_Ba_fe','La_fe','e_La_fe','Zr_fe', 'e_Zr_fe','Y_fe','e_Y_fe',
            'program'
           ]
    new_df = new_df[cols]
    new_df.sort_values(by=['RG_id'],inplace=True)
    
    new_df = new_df.round(2)
    return new_df

df_p112 = combine_table(obj_dir_p112,program=112)
df_p113 = combine_table(obj_dir_p113,program=113)


In [8]:
p112_galah = pd.merge(galah[['sobject_id','logg','teff']], df_p112,left_on='sobject_id', right_on='galah_id', how='right')
p113_galah = pd.merge(galah[['sobject_id','logg','teff']], df_p113,left_on='sobject_id', right_on='galah_id', how='right')


In [13]:
df_both = pd.concat([p112_galah, p113_galah])
df_both = pd.merge(df_both, galah[['sobject_id','ra_dr2','dec_dr2','dr3_source_id']],on='sobject_id')
df_both = pd.merge(df_both, galah_rv[['sobject_id','MJD_local','rv_obst','rv_sme_v2','rv_nogr_obst','e_rv_nogr_obst','rv_galah','e_rv_galah','dr2_radial_velocity','use_rv_flag']],on='sobject_id')
df_both[['rv_obst','rv_nogr_obst','rv_sme_v2','rv_galah','use_rv_flag']]


,rv_obst,rv_nogr_obst,rv_sme_v2,rv_galah,use_rv_flag
0,53.551998,53.576000,53.732314,53.551998,0
1,NaN,NaN,29.725057,29.725057,1
2,49.868000,49.931999,49.949126,49.868000,0
3,99.723000,99.746002,99.593048,99.723000,0
4,-4.131000,-4.051000,-4.090244,-4.131000,0
5,3.119000,3.269000,3.496898,3.119000,0
6,23.688000,23.768999,24.118518,23.688000,0
7,5.624000,5.759000,5.588855,5.624000,0
8,25.080000,25.101000,25.801563,25.080000,0
9,-54.818001,-54.791000,-55.053827,-54.818001,0


In [15]:
df_both.keys()

Index(['sobject_id', 'logg', 'teff', 'RG_id', 'galah_id', 'obj_name',
       'esp_time', 'esp_rv', 'esp_rv_err', 'Li_val', 'e_Li_val', 'vbroad',
       'e_vbroad', 'fe_h', 'ruwe_dr2', 'Ba_fe', 'e_Ba_fe', 'La_fe', 'e_La_fe',
       'Zr_fe', 'e_Zr_fe', 'Y_fe', 'e_Y_fe', 'program', 'ra_dr2', 'dec_dr2',
       'dr3_source_id', 'MJD_local', 'rv_obst', 'rv_sme_v2', 'rv_nogr_obst',
       'e_rv_nogr_obst', 'rv_galah', 'e_rv_galah', 'dr2_radial_velocity',
       'use_rv_flag'],
      dtype='object')

In [ ]:
ss

In [ ]:
df_both.head(1)

In [16]:
def convert_mjd_to_jd(mjd):
    t = Time(mjd, format='mjd', scale='local')
    return t.jd

In [29]:
def comb_data(df):
    new_df = {'RG_id':[],'sobject_id':[],'comb_time':[],'comb_rv':[],'comb_rv_err':[]}

    for i, row in df.iterrows():
        if str(row['MJD_local']) == 'nan':
            galah_bmjd = 'nan'
        else:
            galah_bmjd = convert_mjd_to_jd(row['MJD_local'])
        t = row['esp_time']*1
        r = row['esp_rv']*1
        e = row['esp_rv_err']*1

        t.append(galah_bmjd)
        r.append(row['rv_galah'])
        e.append(row['e_rv_galah'])

        mask = [i for i in range(len(t)) if t[i] != 'nan']
        t, r, e = np.array(t),np.array(r),np.array(e)

        new_df['RG_id'].append(row.RG_id)
        new_df['sobject_id'].append(row.sobject_id)
        new_df['comb_time'].append(t[mask])
        new_df['comb_rv'].append(r[mask])
        new_df['comb_rv_err'].append(e[mask])
        
    new_df = pd.DataFrame(new_df)
    return new_df



In [30]:
def clean_up_rvs(df):
    
    new_df = {'sobject_id':[],'comb_time':[],'comb_rv':[],'comb_rv_err':[]}
    
    for i, row in df.iterrows():
        rv_vals = np.array(row.comb_rv)
        yerr    = np.array(row.comb_rv_err)
        time    = np.array(row.comb_time)
        if row.RG_id!=5:
            mask    = (rv_vals >= -100)
            time, rv_vals, yerr = time[mask], rv_vals[mask], yerr[mask]
            
        new_df['sobject_id'].append(row.sobject_id)
        new_df['comb_time'].append(time)
        new_df['comb_rv'].append(rv_vals)
        new_df['comb_rv_err'].append(yerr)
    new_df = pd.DataFrame(new_df)
    return new_df


In [31]:
new_df = comb_data(df_both)
new_df = clean_up_rvs(new_df)

In [32]:
comb_df = pd.merge(df_both, new_df, on='sobject_id')

In [55]:
comb_df.keys()

Index(['sobject_id', 'logg', 'teff', 'RG_id', 'galah_id', 'obj_name',
       'esp_time', 'esp_rv', 'esp_rv_err', 'Li_val', 'e_Li_val', 'vbroad',
       'e_vbroad', 'fe_h', 'ruwe_dr2', 'Ba_fe', 'e_Ba_fe', 'La_fe', 'e_La_fe',
       'Zr_fe', 'e_Zr_fe', 'Y_fe', 'e_Y_fe', 'program', 'ra_dr2', 'dec_dr2',
       'dr3_source_id', 'MJD_local', 'rv_obst', 'rv_sme_v2', 'rv_nogr_obst',
       'e_rv_nogr_obst', 'rv_galah', 'e_rv_galah', 'dr2_radial_velocity',
       'use_rv_flag', 'comb_time', 'comb_rv', 'comb_rv_err'],
      dtype='object')

In [69]:
all_dfs = []

for i, row in comb_df.iterrows():
    N      = np.array(row.esp_time)
    ids    = [row.sobject_id]*len(N)
    source = ['ESPRESSO']*len(N)
    
    dat1   = pd.DataFrame({'GALAH_ID':ids,
                        'time':np.array(row.esp_time).astype(str), 
                        'rv':np.array(row.esp_rv).astype(str),  
                        'rv_err':np.array(row.esp_rv_err).astype(str),
                        'source':source
                       })

    N      = np.array([row.rv_galah])
    ids    = [row.sobject_id]*len(N)
    source = ['GALAH']*len(N)
    
    dat2   = pd.DataFrame({'GALAH_ID':ids,
                        'time':np.array(row.MJD_local).astype(str), 
                        'rv':np.array(row.rv_galah).astype(str),  
                        'rv_err':np.array(row.e_rv_galah).astype(str),
                        'source':source
                       })
    
    dat = pd.concat([dat1, dat2])
    all_dfs.append(dat)


In [136]:
#pd.concat(all_dfs).to_csv('rv_table.csv',index=False)

In [137]:

comb_df_tb = Table.from_pandas(comb_df)
comb_df_tb['dr3_source_id'] = comb_df_tb['dr3_source_id'].astype(int)
comb_df_tb[['sobject_id','dr3_source_id']]


sobject_id,dr3_source_id
int64,int64
150107004201104,5632958769593943424
151230003202196,5483084721864794752
161013005401317,3142806554658429952
161104004801189,2945063305416392192
150829004301285,6579575475802939392
160129003601260,2899390245231066880
140311007101261,5366629501135968512
170514002101318,5232071641786703488
140806001701013,5909827209206002688


In [110]:



def convert_time_to_mjd(time):
    from datetime import datetime
    from astropy.time import Time

    # Create a datetime object
    dt = datetime.strptime(time, "%Y%m%d")

    # Convert the datetime object to an astropy Time object
    t = Time(dt, format='datetime', scale='utc')

    # Access the Modified Julian Date (MJD)
    mjd = t.mjd
    return mjd

In [94]:
rave = Table.read('/Users/maryumsayeed/Downloads/rave_crossmatch.fits')




In [122]:
from astropy.coordinates import SkyCoord, EarthLocation
rave_loc = EarthLocation.of_site('AAO')
rave_loc

<EarthLocation (-4680888.60272112, 2805218.44653429, -3292788.0804506) m>

In [144]:
joined = join(rave,comb_df_tb,keys_left='source_id',keys_right='dr3_source_id')#
joined['rave_obsdate'] = [i.split('_')[0] for i in joined['rave_obs_id']]
joined['rave_mjd']     = [convert_time_to_mjd(i) for i in joined['rave_obsdate']]
joined['source']       = ['RAVE']*len(joined)

rave_joined = pd.DataFrame({'GALAH_ID':joined['sobject_id'] ,
                            'time':joined['rave_mjd'] ,
                            'rv':joined['hrv_sparv'] ,
                            'rv_err':joined['hrv_error_sparv'] ,
                            'source':joined['source'] ,
                           })

esp_galah = pd.concat(all_dfs)
esp_galah_rave = pd.concat([esp_galah,rave_joined])
esp_galah_rave.sort_values(by=['GALAH_ID','source'])


,GALAH_ID,time,rv,rv_err,source
0,140311007101261,2460388.49997333,23.5296422600186,0.0497316920164264,ESPRESSO
1,140311007101261,2460303.83889431,23.3061153528581,0.0188699131292371,ESPRESSO
2,140311007101261,2460344.87493264,23.2354793778402,0.0516786488909455,ESPRESSO
0,140311007101261,56727.55078125,23.687999725341797,0.0820000022649765,GALAH
0,140707003101315,2460473.87834479,-44.4442597984319,0.050241029908662,ESPRESSO
...,...,...,...,...,...
1,171003002601363,2460467.92056254,-33.5997486540806,0.0326921304472577,ESPRESSO
2,171003002601363,2460500.90336201,-33.7081848901936,0.0492143299110572,ESPRESSO
3,171003002601363,2460443.90797462,-33.5576465070852,0.0329745456465757,ESPRESSO
4,171003002601363,2460427.904157,-33.5284713033847,0.0401656022170961,ESPRESSO


In [132]:
np.mean(joined['hrv_error_sparv'])

1.001

In [131]:

sc = SkyCoord(ra=237.891083333333*u.deg, dec=-69.7031388888889*u.deg)
joined
heliocorr = sc.radial_velocity_correction('heliocentric', obstime=Time('2011-5-25'), location=rave_loc)
print(heliocorr.to(u.km/u.s))
barycorr = sc.radial_velocity_correction(obstime=Time('2011-5-25'), location=rave_loc)
print(barycorr.to(u.km/u.s))

3.797462054511903 km / s
3.807898483229631 km / s


In [99]:
rave.keys()
rave['rave_obs_id']


20030420_1051m27_034
20030420_1051m27_060
20030420_1051m27_082
20030420_1051m27_085
20030420_1051m27_102
20030420_1204m33_015
20030420_1204m33_065
20030420_1204m33_088
20030420_1204m33_093
20030420_1204m33_118
20030420_1204m33_121


In [ ]:
ss

In [ ]:
def make_plot(df):
    plt.figure(figsize=(8,30))
    for i, row in df.iterrows():
        rv_vals = np.array(row.comb_rv)
        yerr    = np.array(row.comb_rv_err)
        time    = np.array(row.comb_time)

        t2 = Time(time, format='jd')

        bmjd2 = t2.tcb.mjd

        bmjd2 = bmjd2 - min(bmjd2)
        plt.subplot(17,2,i+1)
        plt.scatter(bmjd2,rv_vals)

        peak = max(rv_vals) - min(rv_vals)
        plt.title('RG%s (%s) - %.1f'%(row.RG_id, row.program, peak),fontsize=8)
    plt.tight_layout()
    plt.show()
make_plot(comb_df)

In [ ]:
# comb_df.to_csv('comb_rvs.csv',index=False)

In [ ]:
cols = ['sobject_id', 'dr3_source_id','ra_dr2', 'dec_dr2','obj_name','logg', 'teff',  'Li_val', 'vbroad', 'fe_h', 
        'ruwe_dr2']
# both[cols].to_csv('espresso_targets.csv',index=False)